### Load the data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

wine = pd.read_csv('Data/winequality-red.csv', delimiter = ';')
wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [2]:
train, test = train_test_split(wine, test_size = 0.2, random_state = 0)

In [3]:
import statsmodels.api as sm

X_train = train.drop(['quality'], axis=1)
X_train = sm.add_constant(X_train)
y_train = train['quality']

X_test = test.drop(['quality'], axis=1)
X_test = sm.add_constant(X_test)
y_test = test['quality']

### Best Subset Selection 

In [4]:
import statsmodels.api as sm
import itertools
import time
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold

def processSubset(feature_set):
    scaler = StandardScaler()
    log_reg = LogisticRegression(random_state=0, max_iter=10000)
    # Use Stratified K-Fold to maintain the ratio of classes in each fold
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=0)
    pipe = Pipeline([('scaler', scaler), ('log_reg', log_reg)])
    scores = cross_val_score(pipe, X_train[list(feature_set)], y_train, cv=skf, scoring='accuracy')
    return {"model": pipe, "score": np.mean(scores), "features": feature_set}

def getBest(k):
    tic = time.time()
    results = []
    for combo in itertools.combinations(X_train.columns, k):
        results.append(processSubset(combo))
    models = pd.DataFrame(results)
    best_model = models.loc[models['score'].idxmax()]
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", toc - tic, "seconds.")

    return best_model

# Run model selection with cross-validation
models_cv = pd.DataFrame(columns=["score", "model", "features"])
tic = time.time()
for i in range(1, len(X_train.columns) + 1): 
    models_cv.loc[i] = getBest(i)
toc = time.time()

Processed 12 models on 1 predictors in 0.31438517570495605 seconds.
Processed 66 models on 2 predictors in 1.9933438301086426 seconds.
Processed 220 models on 3 predictors in 7.309293031692505 seconds.
Processed 495 models on 4 predictors in 18.219524145126343 seconds.
Processed 792 models on 5 predictors in 31.55221915245056 seconds.
Processed 924 models on 6 predictors in 39.3025689125061 seconds.
Processed 792 models on 7 predictors in 35.946675062179565 seconds.
Processed 495 models on 8 predictors in 23.248604774475098 seconds.
Processed 220 models on 9 predictors in 10.959013223648071 seconds.
Processed 66 models on 10 predictors in 3.598240852355957 seconds.
Processed 12 models on 11 predictors in 0.6809582710266113 seconds.
Processed 1 models on 12 predictors in 0.05989384651184082 seconds.


In [5]:
# Identify the best model
best_index = models_cv['score'].idxmax()
best_model = models_cv.loc[best_index]

print("Best features:", best_model['features'])

Best features: ('fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'sulphates', 'alcohol')


In [6]:
predictors_bs = list(best_model['features'])

scaler = StandardScaler()
log_reg = LogisticRegression(random_state=0, max_iter=10000)
pipe = Pipeline([('scaler', scaler), ('log_reg', log_reg)])
pipe.fit(X_train[predictors_bs], y_train)

train_pred = pipe.predict(X_train[predictors_bs])
test_pred = pipe.predict(X_test[predictors_bs])

In [7]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, roc_auc_score

print("Training Data Metrics:")
print("Confusion Matrix:")
print(confusion_matrix(y_train, train_pred))
print("\nClassification Report:")
print(classification_report(y_train, train_pred))

print("Testing Data Metrics:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, test_pred))
print("\nClassification Report:")
print(classification_report(y_test, test_pred))

Training Data Metrics:
Confusion Matrix:
[[  2   0   5   1   0   0]
 [  0   1  28  12   1   0]
 [  2   0 405 133   6   0]
 [  0   0 165 290  41   0]
 [  0   0  11  98  63   0]
 [  0   0   0   9   6   0]]

Classification Report:
              precision    recall  f1-score   support

           3       0.50      0.25      0.33         8
           4       1.00      0.02      0.05        42
           5       0.66      0.74      0.70       546
           6       0.53      0.58      0.56       496
           7       0.54      0.37      0.44       172
           8       0.00      0.00      0.00        15

    accuracy                           0.59      1279
   macro avg       0.54      0.33      0.35      1279
weighted avg       0.60      0.59      0.58      1279

Testing Data Metrics:
Confusion Matrix:
[[  0   0   2   0   0   0]
 [  0   0   6   4   1   0]
 [  0   0 104  30   1   0]
 [  0   0  39  88  15   0]
 [  0   0   2  15  10   0]
 [  0   0   0   1   2   0]]

Classification Report:
  

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [8]:
train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)
train_f1 = f1_score(y_train, train_pred, average='weighted')
test_f1 = f1_score(y_test, test_pred, average='weighted')

train_prob = pipe.predict_proba(X_train[predictors_bs])
test_prob = pipe.predict_proba(X_test[predictors_bs])
train_auc = roc_auc_score(y_train, train_prob, multi_class="ovo", average="weighted")
test_auc = roc_auc_score(y_test, test_prob, multi_class="ovo", average="weighted")

print(f"Training Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Training F1-Score: {train_f1}")
print(f"Test F1-Score: {test_f1}")
print(f"Training AUC: {train_auc}")
print(f"Test AUC: {test_auc}")

Training Accuracy: 0.5949960906958561
Test Accuracy: 0.63125
Training F1-Score: 0.5768178358443645
Test F1-Score: 0.61375
Training AUC: 0.8041815471040774
Test AUC: 0.732471752442263
